An ipywidget interface to move incorrect predictions within the training dataset should they be labeled wrong

In [2]:
import sys

sys.path.insert(0, "..")
import itertools

import ipywidgets
import matplotlib.pyplot as plt
import numpy as np

import cocpit.config as config
import cocpit.gui as gui

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
plt_params = {
    "axes.labelsize": "xx-large",
    "axes.titlesize": "xx-large",
    "xtick.labelsize": "xx-large",
    "ytick.labelsize": "xx-large",
    "legend.title_fontsize": 12,
}
plt.rcParams["font.family"] = "serif"
plt.rcParams.update(plt_params)

### check classifications from specific model and validation dataloader

In [49]:
torch.cuda.empty_cache()

In [26]:
"""
In gui.py, uses a model and validation dataloader to check predictions.
Loops through all validation loader predictions but only saves incorrect predictions.
The incorrect predictions are loaded into a gui so that a user can decide 
whether the label is wrong (i.e., the model is right)
The image is automatically moved upon choosing a class from the dropdown menu.
"""

top_k_preds = 9  # the top k predictions will be displayed in bar chart
folds = 5  # if kfold cross validation was not used, change to 0

lp = gui.LoaderPredictions()

try:

    # loop over folds and model to get incorrect predictions
    # includes a validation dataloader per fold to get all of the labeled images
    # (i.e., not just 20% at once with a random shuffle)
    for fold in range(folds):
        lp.get_dataloader(fold)
        lp.get_model(fold)
        for batch_idx, ((imgs, labels, paths), index) in enumerate(lp.val_loader):
            g = gui.BatchPredictions(lp, imgs, labels, paths)
            g.find_wrong_indices()

            # make sure there is an incorrect prediction in this batch
            # otherwise skip appending
            if len(wrong_idx) != 0:
                g.top_k_preds(top_k_preds)
                g.append_preds(lp)


except FileNotFoundError:
    print(
        "There are files in the dataloader that have already moved and cannot be found."
    )
    print(
        "This is likely due to running an old model that has not captured the updated file movement."
    )
    print(
        "Try rerunning the model to update the validation dataloaders.  Stopping prematurely."
    )
    pass

lp.concatenate_loader_vars()

print("DONE FINDING INCORRECT PREDICTIONS!")
print(f"There are {len(all_labels)} images to check!")

There are files in the dataloader that have already moved and cannot be found.
This is likely due to running an old model that has not captured the updated file movement.
Try rerunning the model to update the validation dataloaders.  Stopping prematurely.
DONE FINDING INCORRECT PREDICTIONS!
There are 571 images to check!


In [ ]:
"""if you stopped the previous cell early because there are
so many wrong predictions, run this to capture labels up until you stopped waiting"""
lp.concatenate_loader_vars()

In [27]:
# to only look between specific categories run this cell
label_list = dict(zip(config.CLASS_NAMES, np.arange(0, len(config.CLASS_NAMES))))
print(label_list)
# change these two lines to focus on wrong predictions from a specific category
# or box within the confusion matrix
# makes labeling easier focusing on two at a time
human_label = label_list["agg"]
model_label = label_list["rimed"]

lp.hone_incorrect_predictions(label_list, human_label, model_label)

{'agg': 0, 'budding': 1, 'bullet': 2, 'column': 3, 'compact_irreg': 4, 'fragment': 5, 'planar_polycrystal': 6, 'rimed': 7, 'rimed_column': 8, 'sphere': 9}
78 wrong predictions between agg and rimed


In [61]:
"""
code for ipywidget buttons called cocpit/gui.py

this cell displays a bar chart of predictions that the model outputs
a dropdown menu is available to move the image if you think the model got the label right 
when you choose an option from the dropdown list, the image will be moved to that category in the training dataset
if you don't want to move the image and the human labeled correctly, simply click "Next"
"""
gui = gui.GUI(
    all_labels[wrong_trunc],
    all_paths[wrong_trunc],
    all_topk_probs[wrong_trunc],
    all_topk_classes[wrong_trunc],
    all_max_preds[wrong_trunc],
)
gui.make_buttons()
display(ipywidgets.HBox([gui.center, gui.menu, gui.forward]))

In [52]:
gui.index

317